In [1]:
import cv2                                # state of the art computer vision algorithms library
import numpy as np                        # fundamental package for scientific computing
import matplotlib.pyplot as plt           # 2D plotting library producing publication quality figures
import pyrealsense2 as rs                 # Intel RealSense cross-platform open-source API
import pyntcloud
from pyntcloud import PyntCloud
import scipy
from sklearn.neighbors import NearestNeighbors
print("Environment Ready")

%matplotlib
%load_ext autoreload
%autoreload 2

Environment Ready
Using matplotlib backend: MacOSX


In [6]:
pipe = rs.pipeline()
cfg = rs.config()
cfg.enable_device_from_file("data/shadow_with_objects.bag")
profile = pipe.start(cfg)

# Skip 5 first frames to give the Auto-Exposure time to adjust
for x in range(120):
  pipe.wait_for_frames()
  
# Store next frameset for later processing:
frameset = pipe.wait_for_frames()
color_frame = frameset.get_color_frame()
depth_frame = frameset.get_depth_frame()

# Cleanup:
pipe.stop()
print("Frames Captured")

Frames Captured


In [5]:
colorizer = rs.colorizer()
color = np.asanyarray(color_frame.get_data())
align = rs.align(rs.stream.color)
frameset = align.process(frameset)
# Update color and depth frames:
aligned_depth_frame = frameset.get_depth_frame()
colorized_depth = np.asanyarray(colorizer.colorize(aligned_depth_frame).get_data())

# Show the two frames together:
images = np.hstack((color, colorized_depth))
plt.imshow(images)

depth_scale = profile.get_device().first_depth_sensor().get_depth_scale()
depth = np.asanyarray(aligned_depth_frame.get_data())